# Overview

Neural networks comprise of layers/modules that perform operations on data. The `torch.nn` namespace provides all the building blocks you need to build your own neural networks. Every module in PyTorch subclasses the `nn.Module`. A neural network is a module itself that consists of other modules(layers). This nested structure allows for building and managing complex architectures easily.


# Define the Neural Network

We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements the operatioins on input data in the `forward` method.

In [1]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits

model=NeuralNetwork().to('cuda')
# print the model structure
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

## How to use it?

The use the model, we pass it the input data. This executes the model's forward, along with some background operations.

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.

In [2]:
import torch

X=torch.rand(1, 28, 28, device='cuda')
logits=model(X)
pred_probab=nn.Softmax(dim=1)(logits)
y_pred=pred_probab.argmax(1)
y_pred

tensor([9], device='cuda:0')

# Model Layers

Let's break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size `28*28` and see what happens to it as we pass it through the network.

In [3]:
input_image=torch.rand(3, 28,28)
input_image.size()

torch.Size([3, 28, 28])

## nn.Flatten

We initialize the `nn.Flatten` layer to convert each  2D `28*28` image into a contiguous array of 784 pixel values( the minibatch dimension(at dim=0) is maintained)

In [4]:
flatten=nn.Flatten()
flat_image=flatten(input_image)
flat_image.size()

torch.Size([3, 784])

## nn.Linear

The `linear layer` is a module that applies a linear transformation on the input using it stored weightds and biases

In [5]:
layer1=nn.Linear(in_features=28*28, out_features=20)
hidden1=layer1(flat_image)
hidden1.size()

torch.Size([3, 20])

## nn.ReLU

Non-linear activations are what create the complex mappings between the model's inputs and outptus. They are applied after linear transformations to introduce `nonlinearity`, helping neural networks learn a wide variety of phenomena.

In this model, we use `nn.ReLU` between our linear layers, but there's other activations to introduce non-linearity in your model.

In [6]:
hidden1
hidden1=nn.ReLU()(hidden1)
hidden1

tensor([[0.0000, 0.1771, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3789, 0.0000,
         0.1292, 0.2940, 0.4488, 0.0000, 0.0000, 0.4081, 0.9184, 0.0000, 0.1930,
         0.0000, 0.3340],
        [0.0115, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0445, 0.0547, 0.0000,
         0.0533, 0.1382, 0.6336, 0.0000, 0.0637, 0.6124, 0.5755, 0.0000, 0.4556,
         0.2564, 0.1648],
        [0.0000, 0.0190, 0.0000, 0.0000, 0.0000, 0.2212, 0.0679, 0.1994, 0.0000,
         0.0000, 0.1587, 0.5104, 0.0000, 0.3075, 0.5165, 0.4977, 0.0000, 0.3520,
         0.0000, 0.0000]], grad_fn=<ReluBackward0>)

## nn.Sequential

`nn.Sequential` is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequantial containers to put together a quick network like `seq_modules`.

In [7]:
seq_modules=nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)

input_image=torch.rand(3,28,28)
logits=seq_modules(input_image)

## nn.Softmax

The last linear layer of the neural network return `logits` -raw values in [-infty, infty]- which are passed to the `nn.Softmax` module. The logits are scaled to values [0,1] representing the model's predicted probabilities for each class. `dim` parameter indicates the dimension along which the values must sum to 1.

In [8]:
softmax=nn.Softmax(dim=1)
pred_probab=softmax(logits)

# Model Parameters

Many layers inside a neural network are `parameterized`, i.e. have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's `parameters()` of `named_parameters()` methods.

In [9]:
for name,param in model.named_parameters():
    print(f'Layer:{name} | Size: {param.size()} | Values: {param[:2]}')

Layer:linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0077,  0.0262,  0.0144,  ...,  0.0108, -0.0259, -0.0011],
        [ 0.0314, -0.0058, -0.0080,  ..., -0.0285, -0.0137, -0.0182]],
       device='cuda:0', grad_fn=<SliceBackward0>)
Layer:linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0142, -0.0329], device='cuda:0', grad_fn=<SliceBackward0>)
Layer:linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0404, -0.0252, -0.0311,  ..., -0.0264, -0.0133,  0.0117],
        [-0.0259, -0.0330,  0.0201,  ...,  0.0233, -0.0439, -0.0172]],
       device='cuda:0', grad_fn=<SliceBackward0>)
Layer:linear_relu_stack.2.bias | Size: torch.Size([512]) | Values: tensor([-0.0034, -0.0051], device='cuda:0', grad_fn=<SliceBackward0>)
Layer:linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values: tensor([[-0.0347,  0.0083, -0.0411,  ...,  0.0140, -0.0397, -0.0369],
        [-0.0205, -0.0041, -0.0242,  ...,  0.0097,  0

# Reference

* https://pytorch.org/docs/stable/nn.html
* https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html